---

<div align=center>

# SETUP

</div>

## Config

In [1]:
version = "23w31a"

## Install dependencies

In [2]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


## Import dependencies

In [3]:
import os
import numpy as np
from libs import directory
import urllib

## Global data

In [4]:
datapack_path = os.path.abspath("../datapacks/Bookshelf")

---

<div align=center>

# GENERATORS

</div>

## Block list in storage

### Init

In [5]:
block_list_url = f"https://raw.githubusercontent.com/PixiGeko/Minecraft-generated-data/{version}/minecraft-generated/reports/blocks.json"

class Block:
  def __init__(self, name: str, id: int, blockstates_group: int):
    self.name = name
    self._id = id
    self.blockstates_group = blockstates_group

  def __repr__(self) -> str:
    return f"({self.name}, {self._id}, {self.blockstates_group})"
  

class BlockStates:
  def __init__(self, blockstates: dict, default: bool, _bshash: int):
    self.blockstates = blockstates
    self.flat = f"[{','.join([f'{key}={value}' for key, value in blockstates.items()])}]"
    self.default = default
    self._bshash = _bshash

  def set_group_id(self, id: int):
    self.group_id = id

  def __str__(self):
    return f"(states:{dict(self.blockstates)}, default: {self.default})"
  
  def __repr__(self):
    return self.__str__()

  def __hash__(self) -> int:
    return self._bshash

  def __eq__(self, __value: object) -> bool:
    return hash(self) == hash(__value)
  

class BlockStatesGroup:
  def __init__(self, blockstates, id: int):
    self.blockstates = blockstates
    for bs in self.blockstates:
      bs.set_group_id(id)
    self.id = id

  def __hash__(self) -> int:
    return hash("".join(map(lambda bs: str(hash(bs)), self.blockstates)))

  def __str__(self) -> str:
    return self.__repr__()

  def __repr__(self) -> str:
    return f"[ {', '.join(map(lambda bs: bs.__repr__(), self.blockstates))} ]"  

  def __eq__(self, __value: object) -> bool:
    return self.__hash__() == __value.__hash__()
  
class BlockGroup:
  def __init__(self, block_state_group_id: int, properties: list[str]):
    self.block_state_group_id = block_state_group_id
    self.blocks = []
    self.properties = properties
  
  def add_block(self, block_name: str):
    self.blocks.append(block_name)

# Get raw block list
with urllib.request.urlopen(block_list_url) as response:
    blocks_dict = eval(str(response.read())[2:-1].replace("\\n", ""), {'true': True, 'false': False})


# Convert raw list to State & Block lists
block_list: list[Block] = []
state_list: list[BlockStatesGroup] = []
block_group_list: list[BlockGroup] = []
id = 0
for block in blocks_dict.keys():
    group = []
    # The hash will allow the identification of identical blockstates groups
    hash_ = "0"
    if "properties" in blocks_dict[block]:
      props = []
      for (key, property) in blocks_dict[block]["properties"].items():
        property.sort()
        props.append(key + "".join(property))
      props.sort()
      hash_ = hash("".join(props))


    block_id = -1
    block_name = block
    for block in blocks_dict[block]["states"]:
      if "default" in block and block["default"]:
        block_id = block["id"]
      if "properties" in block:
        group.append(BlockStates(block["properties"], block["default"] if "default" in block else False, hash_))
    blockstate_group = BlockStatesGroup(group, id)
    if blockstate_group not in state_list:
      state_list.append(blockstate_group)
      properties = blockstate_group.blockstates[0].blockstates.keys() if len(blockstate_group.blockstates) > 0 else []
      block_group_list.append(BlockGroup(id, properties))
      id += 1
    blockstates_group_id = state_list.index(blockstate_group)
    block_group_list[blockstates_group_id].add_block(block_name)
    block_list.append(Block(block_name, block_id, blockstates_group_id))

block_list.sort(key=lambda block: block._id)

# Keep only ambigious groups, i.e. groups where all state properties are also into other groups
block_group_dict: dict[str, BlockGroup] = dict()
for group in block_group_list:
  for group2 in block_group_list:
    if group.properties != group2.properties:
      if all(prop in group2.properties for prop in group.properties):
        block_group_dict[group.block_state_group_id] = group



### Block array

In [6]:
blockstates_group_nbt_format = """{ \\
      name: "%%block%%", \\
      block_states_group: %%blockstates_group_id%% \\
    }"""

command = """
  data modify storage bs:block block_types set value [ \\
    %%blocks%% \\
  ]
"""


def blockstates_group_to_nbt_object(block: Block) -> str:
    return blockstates_group_nbt_format \
            .replace("%%block%%", block.name) \
            .replace("%%blockstates_group_id%%", str(block.blockstates_group))
    

# Format command
formated_block_list = [blockstates_group_to_nbt_object(block) for block in block_list]

cmd = command.replace("%%blocks%%", ", \\\n    ".join(formated_block_list))

# Write command in the mcfunction file
with open(os.path.join(datapack_path,"data/bs.block/functions/import_block_types_list.mcfunction"), "w") as file:
    file.write("# This file is automatically generated and will most likely be overwritten, do not edit it\n")
    file.write(cmd)

### Blockstates array

In [7]:
blockstates_nbt_format = """{ \\
      group_id: %%group_id%%, \\
      group_start_index: %%group_start%%, \\
      default: %%default%%, \\
      stringified: "%%string%%", \\
      block_state: { \\
        %%blockstate%% \\
      } \\
    }"""

command = """
  data modify storage bs:block block_states set value [ \\
    %%blockstates%% \\
  ]
"""

def value_to_nbt_value(value: str) -> any:
    if value.lower() == "true" or value.lower() == "false":
        return value.lower()
    else:
        return f'"{value}"'


def blockstates_to_nbt_object(blockstates: BlockStates, group_id: int, group_start: int) -> str:
    return blockstates_nbt_format \
            .replace("%%group_id%%", str(group_id)) \
            .replace("%%group_start%%", str(group_start)) \
            .replace("%%default%%", str(blockstates.default).lower()) \
            .replace("%%string%%", blockstates.flat) \
            .replace("%%blockstate%%", ", \\\n          ".join([f'"{key}": {value_to_nbt_value(value)}' for (key, value) in blockstates.blockstates.items()]))

def blockstates_group_to_nbt_object(group: BlockStatesGroup) -> str:
    return blockstates_group_nbt_format.replace("%%blockstates%%", ", \\\n  ".join([blockstates_to_nbt_object(blockstates) for blockstates in group.blockstates]))

formated_blockstates = []

for group in state_list:
    group_start = len(formated_blockstates)
    formated_blockstates += [blockstates_to_nbt_object(bs, group.id, group_start) for bs in group.blockstates]

cmd = command.replace("%%blockstates%%", ", \\\n    ".join(formated_blockstates))

# Write command in the mcfunction file
with open(os.path.join(datapack_path,"data/bs.block/functions/import_block_states_list.mcfunction"), "w") as file:
    file.write("# This file is automatically generated and will most likely be overwritten, do not edit it\n")
    file.write(cmd)

### Block to ID

In [10]:
import math
import json
from typing import Callable, TypeVar
import re

predicate_format = """
  [
    {
      "condition": "minecraft:any_of",
      "terms": [
        %%terms%%
      ]
    }
  ]
"""

predicate_block_format = """
  {
    "condition": "minecraft:location_check",
    "predicate": {
      "block": {
        "blocks": [
          "%%block%%"
        ]
      }
    }
  }
"""

predicate_state_format = """
  {
    "condition": "minecraft:location_check",
    "predicate": {
      "block": {
        %%blocks%%"state": {
          %%blockstates%%
        }
      }
    }
  }
"""

block_tag_format = """
  {
    "values": [
      %%blocks%%
    ]
  }
"""

get_command_format = "execute if predicate %%path%%/group_%%group%% run scoreboard players add @s %%scoreboard%% %%id%%"

block_group_tag_path = "data/bs.block/tags/get/group_%%group%%.json"

def path_to_mcpath(path: str) -> str:
   regex = r"data\/([a-z-_0-9.]+)\/[a-z-_0-9.]+\/((.*)+)[.\/].*"
   groups = re.findall(regex, path)
   return f"{groups[0][0]}:{groups[0][1]}"

def block_to_predicate(block: Block) -> str:
    return predicate_block_format \
            .replace("%%block%%", block.name)

def state_to_predicate(state) -> str:
    predicate = predicate_state_format \
            .replace("%%blockstates%%", ", \n".join([f'"{key}": "{value}"' for (key, value) in state.blockstates.items()]))
    if state.group_id in block_group_dict:
      path = path_to_mcpath(block_group_tag_path.replace("%%group%%", str(state.group_id)))
      predicate = predicate.replace("%%blocks%%", f'"tag": "{path}",\n')
    else:
      predicate = predicate.replace("%%blocks%%", "")
    return predicate

def create_groups(lst: list) -> list:
# The number of groups is the closest power of two that is lower or equal to the size the number of values + 1 to include the group 0
  nb_groups = math.floor(math.log(len(lst), 2)) + 1
  groups = [[] for _ in range(nb_groups)]

  # Compute in which groups the value will be
  for i, value in enumerate(lst):
    quotient = i
    group = 0
    while quotient > 0:
      reminder = quotient % 2
      quotient //= 2
      if reminder == 1:
        groups[group].append(value)
      group += 1
  
  return groups


T = TypeVar("T")
def stringify_groups(groups: list[T], stringify_fun: Callable[[T], str]) -> list[str]:
  formatted_groups = []
  for group in groups:
    formatted_group = ",".join([stringify_fun(value) for value in group])
    predicate = predicate_format.replace("%%terms%%", formatted_group)
    formatted_groups.append(json.dumps(json.loads(predicate), indent=2))
  return formatted_groups

# Extract all states from different BlockStates and flatten the resulting list
states = []
for sub in state_list:
  states += sub.blockstates

def generate_groups(groups: list[str], stringify_fun: Callable[[T], str], path_groups: str, path_function: str, scoreboard: str, function_commands_tail: str):
  groups = create_groups(groups)
  formated_groups = stringify_groups(groups, stringify_fun)

  for i, array in enumerate(formated_groups):
    with open(os.path.join(datapack_path, f"{path_groups}group_{i}.json"), "w") as file:
      file.write(array)
   
  command_format = get_command_format.replace("%%path%%", path_to_mcpath(path_groups)) \
                                     .replace("%%scoreboard%%", scoreboard)

  get_commands = "\n".join([command_format.replace("%%group%%", str(group)).replace("%%id%%", str(math.floor(math.pow(2, group)))) for group in range(len(groups))])

  with open(os.path.join(datapack_path, f"{path_function}.mcfunction"), "w") as file:
    file.write("# This file is automatically generated and will most likely be overwritten, do not edit it\n")
    file.write(f"scoreboard players set @s {scoreboard} 0\n")
    file.write(get_commands)
    file.write("\n\n")
    file.write(function_commands_tail)

for (key, value) in block_group_dict.items():
  blocks = ",\n      ".join([f'"{val}"' for val in value.blocks])
  content = block_tag_format.replace("%%blocks%%", blocks)
  with open(os.path.join(datapack_path, block_group_tag_path.replace("%%group%%", str(key))), "w") as file:
    file.write(content)


generate_groups(block_list, block_to_predicate, "data/bs.block/predicates/get/block_type/", "data/bs.block/functions/block_type/from/block", "bs.block_type.id", "function bs.block:block_type/from/score_id\n")
generate_groups(states, state_to_predicate, "data/bs.block/predicates/get/block_state/", "data/bs.block/functions/block_state/from/block", "bs.block_state.id", "function bs.block:block_state/from/score_id\n")


---

# `_` functions

Generate "_" functions that allow to determine which modules are active or not. They also allow to fix the autocompletion for the user.

In [ ]:
exclude_module = ["glib", "glib.core", "minecraft"]

# Generating all "_" functions
for module in os.listdir(data_path):
    if module in exclude_module: continue
    module_type, module_name = module.split(".")
    if not os.path.isdir(module_path.format(m=module)): os.makedirs(module_path.format(m=module))

    with open(f"{module_path.format(m=module)}/_.mcfunction","w") as f:
        msg = {"text":f"[{module} documentation]","color":"dark_aqua","clickEvent":{"action":"open_url","value":f"https://bookshelf.docs.gunivers.net/en/latest/{module_name}.html"},"hoverEvent":{"action":"show_text","contents":"Click to open URL"}}
        f.write(f"tellraw @s [{msg}]\n".replace("'",'"'))
        f.write(f"scoreboard players set {module} glib.activeModule 1")

# Generating glib.core:load.mcfunction
with open(f"{data_path}/glib.core/functions/load.mcfunction", "r") as old, open(f"{data_path}/glib.core/functions/load_tmp.mcfunction", "w") as new:
    for line in old:
        new.write(line)
        if line.startswith("# Module list"): break
    for module in os.listdir(f"{datapack_path}/data"):
        if module in exclude_module: continue
        new.write(f"function {module}:_\n")
os.remove(f"{data_path}/glib.core/functions/load.mcfunction")
os.rename(f"{data_path}/glib.core/functions/load_tmp.mcfunction", f"{data_path}/glib.core/functions/load.mcfunction")

ValueError: not enough values to unpack (expected 2, got 1)

# Menu > Debug

Allow to add/remove tags to debug systems

In [ ]:
import libs.menu as menu

group_tag = "glib.menu.active"

menus = menu.MenuRunner(
    output        = data_path,
    mcfunction_id = "glib.core:menu",
)

###############
# Menu / Main #
###############

exit_glib_menu = str(
    [
        {"text":"\n"*18},
        {
            "text":  " Thank you for using Glibs!\n",
            "color": "dark_aqua",
            "bold":  True,
        },
        {"text":" Share us your creations on twitter! ", "color":"gray"},
        {
            "text":  "@Gunivers_\n",
            "color": "gold",
            "clickEvent": {
                "action": "open_url",
                "value":  "https://twitter.com/Gunivers_",
            },
            "hoverEvent": {
                "action":   "show_text",
                "contents": "Visit our Twitter page",
            }
        }
    ]
).replace("'",'"').replace('True','true')

glib_menu = menu.Menu(
    output        = data_path,
    mcfunction_id = "glib.core:menu/main",
    menu_tag      = "glib.menu",
    group_tag     = group_tag,
    title         = "Glib Menu",
    exit_message  = exit_glib_menu,
)

menus.add_menu(glib_menu)

glib_menu.add_item(
    menu.Menu(
        output   = None, # not a generated menu, just the tag
        menu_tag = "glib.menu.gamerules",
        title    = "Gamerules"
    )
)
glib_menu.add_item(menu.BLANK_LINE)
glib_menu.add_item(
    menu.Menu(
        output=None,
        menu_tag="glib.menu.debug",
        title="Debug"
    )
)
glib_menu.add_item(menu.BLANK_LINE)
glib_menu.add_item(
    menu.Link("Documentation", "https://glibs.rtfd.io")
)
glib_menu.add_item(menu.BLANK_LINE)
glib_menu.add_item(
    menu.Link("Official website", "https://glib.gunivers.net")
)
glib_menu.add_item(menu.BLANK_LINE)
glib_menu.add_item(
    menu.Link("Our Discord", "https://discord.gg/E8qq6tN")
)
glib_menu.add_item(menu.BLANK_LINE)
glib_menu.add_item(
    menu.Link("Support us", "https://discord.gg/E8qq6tN") # TODO the gunivers utip ?
)

glib_menu.build()

################
# Menu / Debug #
################

debug_menu = menu.Menu(
    output        = data_path,
    mcfunction_id = "glib.core:menu/debug",
    title         = "Glib Menu / Debug",
    menu_tag      = "glib.menu.debug",
    group_tag     = group_tag,
    parent        = glib_menu,
)

menus.add_menu(debug_menu)

exclude_module = ["glib", "glib.core", "minecraft"]

debug_menu.add_item(
    menu.Tag("Debug stick", "glib.debug.stick")
)
debug_menu.add_item(menu.BLANK_LINE)

for module_full in os.listdir(data_path):
    if module_full in exclude_module:
        continue
    
    module = module_full.split(".")[1]
    
    module_menu = menu.Menu(
        output        = data_path,
        mcfunction_id = f"glib.core:menu/debug/{module}",
        menu_tag      = f"glib.menu.debug.{module}",
        parent        = debug_menu,
        group_tag     = group_tag,
        title         = f"Glib Menu / Debug / {module}",
        submenu_name  = module,
    )
    
    debug_menu.add_item(module_menu)
    
    menus.add_menu(module_menu)

    for item in os.listdir(module_path.format(m=module_full)):
        path_item = f"{module_path.format(m=module_full)}/{item}"
        if (
            (os.path.isdir(path_item) and os.path.isfile(f"{path_item}.mcfunction"))
            or (os.path.isdir(path_item) and os.path.isfile(f"{path_item}_ata.mcfunction"))
            or (os.path.isdir(path_item) and os.path.isfile(f"{path_item}_tti.mcfunction"))
            or item in ["_.mcfunction", "child", "accuracy", "config", "debug", "global"]
        ):
            continue
        
        if os.path.isfile(path_item):
            module_menu.add_item(
                menu.Tag(
                    item.replace(".mcfunction", ""),
                    f"glib.debug.{module}.{item.replace('.mcfunction','')}"
                )
            )
            
        if os.path.isdir(path_item):
            item_menu = menu.Menu(
                output        = data_path,
                mcfunction_id = f"glib.core:menu/debug/{module}/{item}",
                menu_tag      = f"glib.menu.debug.{module}.{item}",
                group_tag     = group_tag,
                parent        = module_menu,
                title         = f"Glib Menu / Debug / {module} / {item}",
            )
            
            module_menu.add_item(item_menu)

            menus.add_menu(item_menu)
            
            for subitem in os.listdir(path_item):
                if os.path.isfile(f"{path_item}/{subitem}") and subitem != "_.mcfunction":
                    item_menu.add_item(
                        menu.Tag(
                            subitem.replace(".mcfunction",""),
                            f"glib.debug.{module}.{item}.{subitem.replace('.mcfunction','')}"
                        )
                    )
            
            item_menu.build()

module_menu.build()

#############
# Gamerules #
#############

gamerules_menu = menu.Menu(
    output        = data_path,
    mcfunction_id = "glib.core:menu/gamerules",
    menu_tag      = "glib.menu.gamerules",
    group_tag     = group_tag,
    parent        = glib_menu,
    title         = "Glib Menu / Gamerules",
)

menus.add_menu(gamerules_menu)

with open(f"{minecraft_data_path}/data/commands/syntaxes/gamerule.txt") as f:
    for rule in f:
        rulename = rule.split(" ")[1]
        ruletype = rule.split(" ")[3][:-2]
        gamerules_menu.add_item(
            menu.Gamerule(rulename, ruletype)
        )

gamerules_menu.build()

##############
# Build menu #
##############

menus.build()

print("Menus builded successfully!")
print("Now, put the following command in a loop:")
print(f"execute if entity @a[tag={group_tag}] run function {menus.mcfunction_id}")

---

# Refactor

In [ ]:
import os
import regex as re
from multiprocess import Pool, cpu_count
from LRFutils import progress

folders = ["../docs", "../datapacks"]

replace_list = [
    (r"bs.var(?<numero>[0-9])", r"bs.in.\g<numero>"),
    (r"bs.res(?<numero>[0-9])", r"bs.out.\g<numero>"),
    (r"bs.var\[", r"bs.in.["),
    (r"bs.res\[", r"bs.out.["),
    (r"bs.var<", r"bs.in.<"),
    (r"bs.res<", r"bs.out.<"),
    (r"bs.parentId", r"bs.id.parent"),
    (r"bs.targetId", r"bs.id.target"),
    (r"bs.blockId", r"bs.block.id"),
    (r"bs.itemId", r"bs.item.id"),
    (r"bs.vectorX", r"bs.vector.x"),
    (r"bs.vectorY", r"bs.vector.y"),
    (r"bs.vectorZ", r"bs.vector.z"),
    (r"bs.vectorLeft", r"bs.vector.x"),
    (r"bs.vectorUp", r"bs.vector.y"),
    (r"bs.vectorFront", r"bs.vector.z"),
    (r"bs.locX", r"bs.loc.x"),
    (r"bs.locY", r"bs.loc.y"),
    (r"bs.locZ", r"bs.loc.z"),
    (r"bs.oriH", r"bs.ori.h"),
    (r"bs.oriV", r"bs.ori.v"),

]

def replace_in_file(path,file, replace_list):
    import regex as re
    for search, replace in replace_list:
        with open(path + "/" + file, "r+", encoding="cp437") as f:
            content = f.read()
            f.seek(0)
            f.write(re.sub(search, replace, content))
            f.truncate()

nb_files = 0
for folder in folders:
    for path, subdirs, files in os.walk(folder):
        nb_files += len(files)    

n=0
pool = Pool(cpu_count())
bar = progress.Bar(nb_files)

for folder in folders:
    for path, subdirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".mcfunction") or file.endswith(".json") or file.endswith(".txt") or file.endswith(".md") or file.endswith(".rst"):
                pool.apply(replace_in_file, args=(path,file,replace_list))
            n += 1
            bar(n)
pool.close()
pool.join()


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 21351/21351 0:14:21eta 0:00:00
